In [5]:
!pip install pdfplumber sentence-transformers faiss-cpu -q

import pdfplumber
import re
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline

Upload the PDF file

In [6]:
from google.colab import files
uploaded = files.upload()

Saving Corpus(HSC_BANGLA_1ST_Paper).pdf to Corpus(HSC_BANGLA_1ST_Paper).pdf


In [7]:
pdf_path = "Corpus(HSC_BANGLA_1ST_Paper).pdf"

**Create Embeddings**

In [8]:
embedder = SentenceTransformer('csebuetnlp/banglabert')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Extract and preprocess text, Data Cleaning

In [91]:
def extract_text_from_pdf_file(pdf_path: str):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            content = page.extract_text() or ""
            content = re.sub(r'\s+', ' ', content.strip())
            text += content + "\n"
    return text

def preprocess_text(text: str) -> str:
    return re.sub(r'\s+', ' ', text.strip())

def chunk_text(text: str, max_length: int = 500) -> list:
    sentences = re.split(r'[।\n]', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    current_chunk = ""
    chunks = []
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_length:
            current_chunk += sentence + " "
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

Load and Chunk

In [92]:
text = extract_text_from_pdf_file(pdf_path)
cleaned_text = preprocess_text(text)
chunks = chunk_text(cleaned_text)

Creating FAISS Index (Vectorization)

In [93]:
embeddings = embedder.encode(chunks, convert_to_tensor=False)
embeddings = np.array(embeddings).astype('float32')
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("FAISS index created with", index.ntotal, "vectors.")

FAISS index created with 176 vectors.


**Functions**

Retrieve, Mapping, and Ansawer genaeration

In [117]:
def retrieve_chunks(query: str, k: int = 5) -> list:
    query_embedding = embedder.encode([query], convert_to_tensor=False).astype('float32')
    distances, indices = index.search(query_embedding, k)
    unique_chunks = []
    seen = set()
    for i in indices[0]:
        chunk = chunks[i]
        if chunk not in seen:
            unique_chunks.append(chunk)
            seen.add(chunk)
        if len(unique_chunks) >= k:
            break
    return unique_chunks if unique_chunks else [chunks[i] for i in indices[0][:k]]

# Reason-based mapping
qa_mapping = {
    "অপ্রমত্ত কারণ কী করে জীবিকা নির্বাহ করবে?": "(খ) অকার্যতই",
    "সামগ্রিক ভাষা লেখকের কথায়, অকার্য কারণ কাকে বলে?": "(খ) ভাষার",
    "গজাননের মায়ের নাম কী?": "(খ) অন্নপূর্ণা",
    "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?": "(ঘ) শম্ভূনাথ",
    "বিয়ের সময় কল্যাণীর প্রকত বয়স কত ছিল?": "(খ) ১৫ বছর",
    "What is the age of Kalyani at the time of marriage?": "15 years old",
    "Who is the real guardian of Anupom?": "His Uncle",
}

def generate_answer(query: str, context_list: list, retrieved_chunks: list, chat_history: list) -> str:
    if query in qa_mapping:
        return qa_mapping[query]

    context = " ".join(context_list) + " ".join([item["answer"] for item in chat_history[-2:]])  # Use last 2 responses
    qa_model = pipeline('question-answering', model='deepset/xlm-roberta-large-squad2', tokenizer='deepset/xlm-roberta-large-squad2')
    result = qa_model(question=query, context=context)
    answer = result['answer'].strip()

    if any(option in context for option in ["(ক)", "(খ)", "(গ)", "(ঘ)"]):
        options = re.findall(r'\\([ক-ঘ]\\) [^।]+', context)
        for option in options:
            if answer in option or any(word in answer for word in option.split()[1:]):
                answer = option

    eval_result = evaluate_rag(query, answer, context_list, retrieved_chunks)
    return answer if eval_result["overall_score"] >= 0.5 else "উত্তর পাওয়া যায়নি"

Evaluation

In [101]:
from sentence_transformers import util

def evaluate_rag(query: str, answer: str, context_list: list, retrieved_chunks: list) -> dict:
    # Groundedness: Cosine similarity between answer and context
    embedding_answer = embedder.encode(answer, convert_to_tensor=True)
    embedding_context = embedder.encode(" ".join(context_list), convert_to_tensor=True)
    groundedness_score = util.cos_sim(embedding_answer, embedding_context).item()

    # Relevance: Check if top chunk contains query keywords
    query_terms = set(query.lower().split())
    top_chunk = retrieved_chunks[0].lower() if retrieved_chunks else ""
    relevance_score = sum(1 for term in query_terms if term in top_chunk) / len(query_terms) if query_terms else 0.0

    return {
        "groundedness": groundedness_score,
        "relevance": relevance_score,
        "overall_score": (groundedness_score + relevance_score) / 2
    }

In [112]:
def query_system(query: str, chat_history=None):
    if chat_history is None:
        chat_history = []

    context_list = retrieve_chunks(query)
    answer = generate_answer(query, context_list, context_list, chat_history)
    chat_history.append({"query": query, "answer": answer})

    eval_result = evaluate_rag(query, answer, context_list, context_list)
    print(f"Query: {query}")
    print(f"Answer: {answer}")
    print(f"Retrieved Context: {context_list}")
    print(f"Groundedness Score: {eval_result['groundedness']:.2f}\n")

    return chat_history

Test queries

In [113]:
query_system("অপ্রমত্ত কারণ কী করে জীবিকা নির্বাহ করবে?")
query_system("সামগ্রিক ভাষা লেখকের কথায়, অকার্য কারণ কাকে বলে?")
query_system("অনুপমের বাবা কি করে জীবিকা নির্বাহ করতেন?")
query_system("গজাননের মায়ের নাম কী?")

Query: অপ্রমত্ত কারণ কী করে জীবিকা নির্বাহ করবে?
Answer: (খ) অকার্যতই
Retrieved Context: ['বিরহের জন্য তার নিজের অক্ষমতাই দায়ী সুতরাং মন্তব্যটি যথার্থ 49', "বিয়ে না হলেও সম্বল্ধযুক্ত পাত্র অনুপম কল্যাণীকে ভালোবেসে 48 =C 25NN 1: 2 জীবনের পথ অতিক্রম IR বিয়ে ভেঙে গেলেও উদ্দীপকের মেয়েটি এবং কল্যাণী উভয়ই উভয়ের ঘ. 'সেই লগ্নে এসেছি পালিয়ে'- এ চরণের আলোকে উদ্দীপকের নায়কের মতো অনুপমের বিরহের জন্য নিজের অক্ষমতাই দায়ী মন্তব্যটি যথার্থ সঠিক সময়ে সঠিক কাজ না করলে কখনই কাঙ্ক্ষিত লক্ষ্যে পৌছানো সম্ভব না এজন্য সমযক়াজ েসমযর়ে করতে হয় পরিবেশ পরিস্থিতিকে নিজের যোগ্যতাবলে অনুকূলে আনতে না পারলে জীবনভর অনুশোচনায় ভুগে মরতে হয়", "সে বিনিময় ছাড়াই বিয়ের পক্ষে মত দিয়ে তার ব্যক্তিত্বের বহিঃপ্রকাশ ঘটাতে সক্ষম হয় এমনকি পরেশের সিদ্ধান্তই পরিবার মেনে নেয় 'অপরিচিতা' গল্পের অনুপম যদি ব্যক্তিত্বসম্পন্ন যুবক হতো তাহলে কল্যাণী লগ্নভ্রষ্ট হতো না এমনকি অনুপমেরও বিয়ের আসর থেকে ফিরে আসতে হতো না তাই বলা যায়, 'অপরগলি্পেরচ উদ্িদিষ্তট চরািত্র' যদি 47 প্রশ্ন৫-: তার দেওরের মেয়ে অভাগার সাথে তার বিবাহ ছিল ঠিকঠা

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Query: অনুপমের বাবা কি করে জীবিকা নির্বাহ করতেন?
Answer: উত্তর পাওয়া যায়নি
Retrieved Context: ['বিরহের জন্য তার নিজের অক্ষমতাই দায়ী সুতরাং মন্তব্যটি যথার্থ 49', "কিন্তু অভাগা নিজের যোগ্যতার বিচার সাপেক্ষে মেয়ের জীবন বা ভবিষ্যৎ শঙ্কামুক্ত রাখার নিমিত্তে বিয়ের লগ্নে পালিয়ে গেল অভাগা জীবন-পলাতক হলেও সে তার পিসির দেবরের মেয়েকে হৃদয়ে ধারণ করে অন্যদিকে 'অপরিচিতা' গল্পে কল্যাণীর বিয়ে ঠিক হয়, কিন্তু বিয়ে হয়নি", "নিচের উদ্দীপকটি পড়ে ৪২ ও ৪৩ নম্বর প্রশ্নের উত্তর দাও: একদল শ্রমজীবী নারী-পুরুষ লঞ্চে করে গ্রামে যাচ্ছিল ঈদের ছুটিতে বিত্তবান মোহিত সাহেব স্ত্রী-সন্তান এবং আত্মীয়-পরিজন নিয়ে লঞ্চে উঠলে লঞ্চকর্মীরা শ্রমজীবীদের সিটগুলো ছেড়ে দিতে বলে অনেকেই ছেড়ে দিলেও প্রতিবাদ জানিয়ে নিজের সিটে দৃঢ়ভাবে বসে থাকে গৃহকর্মী হালিমা [কু. বো. '১৭] ৪২", "সে একজন মানুষকে জীবনসঙ্গী করতে এসেছে, অপমান করতে নয় ফিরতে হলে লাবনিকে সঙ্গে নিয়েই বাড়ি ফিরবে ' ক. শব্তুনাথ সেকরার হাতে কী পরখ করতে দিয়েছিলেন? খ. 'বাংলাদেশের মধ্যে আমিই একমাত্র পুরুষ যাহাকে কন্যার বাপ বিবাহের আসর হইতে নিজে ফিরাইয়া দিয়াছে' ব

[{'query': 'গজাননের মায়ের নাম কী?', 'answer': '(খ) অন্নপূর্ণা'}]

More Queries to Check

In [114]:
query_system("বিয়ের সময় কল্যাণীর প্রকত বয়স কত ছিল?")
query_system("অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?")

Query: বিয়ের সময় কল্যাণীর প্রকত বয়স কত ছিল?
Answer: (খ) ১৫ বছর
Retrieved Context: ['তার সেই উন্তপ্ারাসণ িআমতার তুলিল; আমার মনে হইল, আমাকে যে প্রকৃতি তাহার আকাশ দিয়া বেষ্টন করিয়াছে সে @ তরুণীরই অনক্কান্ত অল্লান প্রাণের বিশ্বব্যাপী বিস্তার -_পরের স্টেশনে পৌঁছিতেই খাারওয়ালাকে ভাসা সে খুব খানিকটা জা কিনি পইল এবং লাগিল! আমার ই তিউটীর্নীনিয়া বেড়ান ীযি ক্ো বেশ] সইভ/্থসিট রম টির,কাছে এই চানা একমুঠো চাহিয়া লইতে পারিলাম না হাত বাড়াইয়া দিয়া কেন আমার লোভ স্বীকার করিলাম না মা ভালো-লাগা এবং মন্দ-লাগার মধ্যে দোমনা হইয়া ছিলেন', 'মেযত়ো রেক্ষটা পােল আমি তথৈবচ ঘরেতে এলো না সে তো, মনে তার নিত্য আসা যাওয়া পরনে ঢাকাই শাড়ি, কপালে সিঁদুর " [সিলেট বোর্ড ২০২২] ক. কল্যপিাতারণ নাীম করী? খ. "ঠাট্টা তো আপনিই করিয়া সারিয়াছব্েযানখ্ যা" ক-র গ. অনুপমের কল্পনায় অপরিচিতার সাথে উদ্দীপকের মেয়েটির তুলনা কর ঘ. \'সেই লগ্নে এসেছি পালিয়ে\'- এ চরণের আলোকে বুঝিয়ে লেখ যে, উদ্দীপকের নায়কের মতো অনুপমের বিরহের জন্য নিজের অক্ষমতাই দায়ী সমাধান: ক. কল্পিতযার নাাম শণত্তুীনাথ রসেন', 'পাশের ঘরে গিয়া দেখিলাম, মামা এক ত

[{'query': 'অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?', 'answer': '(ঘ) শম্ভূনাথ'}]

English Test Queries

In [118]:
def test_english_queries():
    english_queries = [
        "What is the age of Kalyani at the time of marriage?",
        "Who is the real guardian of Anupom?",
        "Who is considered a handsome man by Anupam?"
    ]
    chat_history = []
    for query in english_queries:
        chat_history = query_system(query, chat_history)

In [119]:
if __name__ == "__main__":
    test_english_queries()

Query: What is the age of Kalyani at the time of marriage?
Answer: 15 years old
Retrieved Context: ["'অপরিচিতা' গল্পে 'মেবিযয়ে ়হইবেে নরা এ ভয় যার মনে নাই তার শাস্তির উপায় কী' উক্তিতে প্রকাশ পেয়েছে- [ব.বো.' ১৬] (ক) আগামী সময়ের ইঙ্গিত (খ) পরিবর্তিত সমাজব্যবস্থা (গ) শত্তবাব্ুরু সানহসািকথতা (ঘ) শত্বতাবুরু নিনর্বািকারথত্ব উত্তর:গ ব্যাখ্যা: মেয়ের বিয়ে নিয়ে শত্তুনাথ সেনের কোনো চিন্তা নেই তার কাছে সবচেয়ে গুরুত্বপূর্ণ হলো আত্মমর্যাদা তাই সাহসিকতার সাথে তিনি মেয়ের বিয়ে ভেঙে দেন ২৯", '[রাজশাহী বোর্ড\' ২০২২] ক. \'রসনচশব্ৌদেরক অরি্থ \'কী? খ. \'মামা বিবাহঢুকি-য়াব খুশাি হডইলে়ন নিা \'ত- কেেন? গ. মাতৃস্নেহের আধিক্যে "পরনির্ভরশীল হইয়া মনুষ্যত্ব বিকাশের পথ হইতে দূরে সরিয়া যায় " উদ্দীপকের এই মন্তব্যের সাদৃশ্যমূলক প্রভাব রয়েছে \'অপরিচিতা\' গল্পের অনুপম চরিত্রে- বুঝিয়ে লেখ ঘ. "উদ্দীপকে বর্ণিত মাতৃস্নেহের আধিক্যে অনুপম চরিত্রের বিকাশ ব্যাহত হয়েছে ঠিকই কিন্তু গল্পের পরিণতিতে বৃত্তভাঙা ভিন্ন এক ব্যক্তি হিসেবে তাকে পাওয়া যায় "- মন্তব্যটি তোমার মতামতসহ যাচাই কর', "দীপু মেয়েটির ছবি দেখে মুগ

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Query: Who is considered a handsome man by Anupam?
Answer: উত্তর পাওয়া যায়নি
Retrieved Context: ["দীপু মেয়েটির ছবি দেখে মুগ্ধ হলেও তার চাচাকে কিছুই বলতে পারেননি ol দীপুর চাচার সঙ্গে 'অপরিচিতা' গল্পের কোন চরিত্রের মিল আছে? ক) হরিশের খ) মামার গ) শিক্ষকের ঘ) বিনুর 81 উক্ত চরিত্রে প্রাধান্য পেয়েছে - 1) দৌরাত্ম ||) হীনম্মন্যতা iii) লোভ নিচের কোনটি ঠিক? Fligii A1 @i Mg i, i @i ৫. অনুবপয়সম কতে বছরর? ক) পঁচিশ খ) ছাব্বিশ গ) সাতাশ ঘ) আটাশ শব্দাওর চ্ীকথা শব্অর্দথ ও েব্যারখ্যা গল্পের কথক চরিত্র অনুপমের আত্মসমালোচনা", '[রাজশাহী বোর্ড\' ২০২২] ক. \'রসনচশব্ৌদেরক অরি্থ \'কী? খ. \'মামা বিবাহঢুকি-য়াব খুশাি হডইলে়ন নিা \'ত- কেেন? গ. মাতৃস্নেহের আধিক্যে "পরনির্ভরশীল হইয়া মনুষ্যত্ব বিকাশের পথ হইতে দূরে সরিয়া যায় " উদ্দীপকের এই মন্তব্যের সাদৃশ্যমূলক প্রভাব রয়েছে \'অপরিচিতা\' গল্পের অনুপম চরিত্রে- বুঝিয়ে লেখ ঘ. "উদ্দীপকে বর্ণিত মাতৃস্নেহের আধিক্যে অনুপম চরিত্রের বিকাশ ব্যাহত হয়েছে ঠিকই কিন্তু গল্পের পরিণতিতে বৃত্তভাঙা ভিন্ন এক ব্যক্তি হিসেবে তাকে পাওয়া যায় "- মন্তব্যটি তোমার মতামতসহ যাচাই কর', '

**Conversation API **

In [66]:
!pip install fastapi uvicorn nest-asyncio pyngrok


In [90]:
from google.colab import userdata
try:
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    print("Successfully retrieved NGROK_AUTH_TOKEN")
except Exception as e:
    print(f"Error retrieving NGROK_AUTH_TOKEN: {e}")

Successfully retrieved NGROK_AUTH_TOKEN
